In [4]:
import pandas as pd
import os

def get_classfy_num(dataframe, index):
    sort_num = 100
    dataframe.sort_values(by = index, ascending=False, inplace = True)
    dataframe = dataframe.reset_index(drop = True)
    df_bright_sort = dataframe[:sort_num]
    # print(list(df_bright_sort["quantitative_function"]))
    num_1 = 0
    num_105 = 0
    num_11 = 0
    num_6 = 0
    for i in list(df_bright_sort["quantitative_function"]):
        if i > 1:
            num_1 += 1
        if i > 1.05:
            num_105 += 1
        if i > 1.1:
            num_11 += 1
        if i < 0.6:
            num_6 += 1
    # print("calssfy 1: " ,num_1, num_1 / sort_num)
    # print("calssfy 0.9: " ,num_9, num_9 / sort_num)
    # print("calssfy 1.1: " ,num_11, num_11 / sort_num)
    return num_1, num_105, num_11, num_6

model_name = "eUniRep-Augmenting"
sampling_method = "random_1-2_ratio"
top_model_name = "lin"
test_mut_num = 2
training_objectives = f'gfp_SK_test_{test_mut_num}'#'SK_split_test_3
n_train_seqs = '1000'
# seed = 0
step1_bright_degree = 0.6
step2_bright_degree = 1.0

output_path = f"/share/jake/github/low_n_output/method_3/result/{top_model_name}"
for seed in range(5):
    input_path = f"/share/jake/github/low_n_output/predict_qfunc/eUniRep-Augmenting"
    input_csv_name = f"{model_name}_{top_model_name}_{sampling_method}_{training_objectives}_n_train_seqs_{n_train_seqs}_seed_{seed}"
    test_df = pd.read_csv(f"{input_path}/{input_csv_name}.csv")

    step1_recall_1, step1_recall_105, step1_recall_11, step1_predict_less_06 = get_classfy_num(test_df, f"{model_name}_{top_model_name}")
    results_dict = {
            "step 1 recall 1.0": step1_recall_1,
            "step 1 recall 1.05": step1_recall_105,
            "step 1 recall 1.1": step1_recall_11,
            "step 1 recall less 0.6": step1_predict_less_06,
        }
    setp2_test_df = test_df[test_df[f"{model_name}_{top_model_name}"] > step1_bright_degree]
    print("Quantity of remaining parts fo step1(Quantity of step2 test df): ", setp2_test_df.shape[0])
    step1_classfy_06 = setp2_test_df[setp2_test_df["quantitative_function"] > step1_bright_degree].shape[0] / setp2_test_df.shape[0]
    step1_predict_hight = test_df[test_df[f"{model_name}_{top_model_name}"] > step2_bright_degree]
    step1_pre_postive = step1_predict_hight.shape[0]
    print("step1_pre_postive: ", step1_pre_postive, len(list(step1_predict_hight["quantitative_function"])))
    step1_true_postive_num = step1_predict_hight[step1_predict_hight["quantitative_function"] > step2_bright_degree].shape[0]
    step1_true_postive_11_num = step1_predict_hight[step1_predict_hight["quantitative_function"] > 1.1].shape[0]
    step1_false_postive_06_num = step1_predict_hight[step1_predict_hight["quantitative_function"] < 0.6].shape[0]
    if step1_pre_postive == 0:
        step1_true_postive = step1_true_postive_11 = step1_false_postive_06 = 0
    else:
        step1_true_postive = step1_true_postive_num / step1_pre_postive
        step1_true_postive_11 = step1_true_postive_11_num / step1_pre_postive
        step1_false_postive_06 = step1_false_postive_06_num / step1_pre_postive

    results_dict.update({
        "step 1 classfy 0.6": step1_classfy_06,
        "step 1 predict postive": step1_pre_postive,
        "step 1 true postive num": step1_true_postive_num,
        "step 1 true postive": step1_true_postive,
        "step 1 true postive num(>1.1)": step1_true_postive_11_num,
        "step 1 true postive(>1.1)": step1_true_postive_11,
        "step 1 true postive num(>1.1)": step1_true_postive_11_num,
        "step 1 true postive(>1.1)": step1_true_postive_11,
        "step 1 false postive num(<0.6)": step1_false_postive_06_num,
        "step 1 false postive(<0.6)": step1_false_postive_06,
    })
    print(results_dict)
    results = pd.DataFrame(columns=sorted(results_dict.keys()))
    results = results.append(results_dict, ignore_index=True)
    acc_save_path = os.path.join(output_path, f"{model_name}_{top_model_name}_{sampling_method}_{training_objectives}_n_train_seqs_{n_train_seqs}_result_recall.csv")
    if os.path.exists(acc_save_path):
        results.to_csv(acc_save_path, mode='a', header=False, index=False,
                columns=sorted(results.columns.values))
    else:
        results.to_csv(acc_save_path, mode='w', index=False,
                columns=sorted(results.columns.values))

Quantity of remaining parts fo step1(Quantity of step2 test df):  9459
step1_pre_postive:  137 137
{'step 1 recall 1.0': 46, 'step 1 recall 1.05': 14, 'step 1 recall 1.1': 4, 'step 1 recall less 0.6': 0, 'step 1 classfy 0.6': 0.9793847129717729, 'step 1 predict postive': 137, 'step 1 true postive num': 62, 'step 1 true postive': 0.45255474452554745, 'step 1 true postive num(>1.1)': 6, 'step 1 true postive(>1.1)': 0.043795620437956206, 'step 1 false postive num(<0.6)': 1, 'step 1 false postive(<0.6)': 0.0072992700729927005}
Quantity of remaining parts fo step1(Quantity of step2 test df):  9388
step1_pre_postive:  198 198
{'step 1 recall 1.0': 58, 'step 1 recall 1.05': 22, 'step 1 recall 1.1': 6, 'step 1 recall less 0.6': 0, 'step 1 classfy 0.6': 0.9751810822326374, 'step 1 predict postive': 198, 'step 1 true postive num': 107, 'step 1 true postive': 0.5404040404040404, 'step 1 true postive num(>1.1)': 7, 'step 1 true postive(>1.1)': 0.03535353535353535, 'step 1 false postive num(<0.6)':